<a href="https://colab.research.google.com/github/ibabedal/BLCA-analysis/blob/main/BLCA_Multi_Omic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Data combind TMB data

In [1]:
!wget https://raw.githubusercontent.com/ibabedal/BLCA-analysis/main/processing-mrna-cna.tar.gz
!wget https://raw.githubusercontent.com/ibabedal/BLCA-analysis/main/DNA-TMB.tar.gz

!tar xf DNA-TMB.tar.gz
!tar xf processing-mrna-cna.tar.gz

--2022-11-09 17:33:27--  https://raw.githubusercontent.com/ibabedal/BLCA-analysis/main/processing-mrna-cna.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21932951 (21M) [application/octet-stream]
Saving to: ‘processing-mrna-cna.tar.gz’

processing-mrna-cna 100%[===================>]  20.92M  --.-KB/s    in 0.1s    

2022-11-09 17:33:28 (153 MB/s) - ‘processing-mrna-cna.tar.gz’ saved [21932951/21932951]

--2022-11-09 17:33:28--  https://raw.githubusercontent.com/ibabedal/BLCA-analysis/main/DNA-TMB.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaitin

In [2]:
!ls -l 

total 345604
-rw-rw-r-- 1  501 staff  23190249 Oct  6 20:58 CNA-TMB.csv
-rw-r--r-- 1  501 staff 171529092 Oct 21 17:54 DNA-TMB.csv
-rw-r--r-- 1 root root   76917479 Nov  9 17:33 DNA-TMB.tar.gz
-rw-rw-r-- 1  501 staff  60317128 Oct  6 21:37 mRNA-TMB.csv
-rw-r--r-- 1 root root   21932951 Nov  9 17:33 processing-mrna-cna.tar.gz
drwxr-xr-x 1 root root       4096 Nov  7 14:37 sample_data


# Installing and Loading needed Libraries

In [34]:
import pandas as pd
import numpy as np
import psutil
import time
import concurrent.futures
import datetime
import os
import sys
from sklearn.preprocessing import MinMaxScaler

In [20]:
!pip install geneticalgorithm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from geneticalgorithm import geneticalgorithm as ga

# Data Engineering the Omics

## CNA

In [22]:
df_cna = pd.read_csv('./CNA-TMB.csv')
df_cna.head()

,SAMPLE_ID,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224,TMB_CLASS
0,TCGA-2F-A9KO-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,2
1,TCGA-2F-A9KP-01,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,1
2,TCGA-2F-A9KQ-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,TCGA-2F-A9KR-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,0,0,0,0,-1,2
4,TCGA-2F-A9KT-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2


In [23]:
x = df_cna.drop(['SAMPLE_ID', 'TMB_CLASS'], axis=1)
y = df_cna[['TMB_CLASS']]

In [24]:
def do_something(x):
    '''
    do stuff
    '''
    pid  = os.getpid()
    ppid = os.getppid()
    start = time.time()
    print("PPID %s->%s Started"%(ppid,pid))
    #df['diff'] = datetime.datetime.now() - pd.to_datetime(df['Order Date'])
    for i in x.columns:
      x[i] = ((x[i] - x[i].min()) + 0.1)
    stop  = time.time()
    completed_in  = round(stop - start,2)
    return(x)

In [25]:
## parallel execution , credit to https://betterprogramming.pub/pandas-how-to-process-a-dataframe-in-parallel-make-pandas-lightning-fast-669978cf5356

logical    = False
df_results = []
num_procs  = psutil.cpu_count(logical=logical)
#if len(sys.argv) > 1:
#    num_procs = int(sys.argv[1])

big_dataframe = x.copy()
splitted_df = np.array_split(big_dataframe, num_procs)
start = time.time()
with concurrent.futures.ProcessPoolExecutor(max_workers=num_procs) as executor:
    results = [ executor.submit(do_something, x=df) for df in splitted_df ]
    for result in concurrent.futures.as_completed(results):
        try:
            df_results.append(result.result())
        except Exception as ex:
            print(str(ex))
            pass
end = time.time()
print("-------------------------------------------")
print("PPID %s Completed in %s"%(os.getpid(), round(end-start,2)))
df_results = pd.concat(df_results)

PPID 75->280 Started
-------------------------------------------
PPID 75 Completed in 281.29


In [26]:
x = df_results.copy()
x.head()

,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,...,2.1,2.1,1.1,2.1,1.1,2.1,2.1,2.1,2.1,1.1
1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,...,2.1,2.1,1.1,2.1,1.1,2.1,2.1,2.1,2.1,1.1
2,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...,1.1,1.1,0.1,1.1,0.1,1.1,1.1,1.1,1.1,0.1
3,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...,1.1,1.1,0.1,1.1,0.1,2.1,2.1,2.1,2.1,0.1
4,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...,1.1,1.1,0.1,1.1,0.1,1.1,1.1,1.1,1.1,0.1


In [27]:
## saving to file to avoid recacluation
x.to_csv('cna_features_shifted.csv',index=False)

In [28]:
def do_something_2(x):
    '''
    do stuff
    '''
    pid  = os.getpid()
    ppid = os.getppid()
    start = time.time()
    print("PPID %s->%s Started"%(ppid,pid))
    #df['diff'] = datetime.datetime.now() - pd.to_datetime(df['Order Date'])
    for i in x.columns:
      sum_feature = x[i].sum()
      x[i] = x[i]/sum_feature
    stop  = time.time()
    completed_in  = round(stop - start,2)
    return(x)

In [29]:
## parallel execution , credit to https://betterprogramming.pub/pandas-how-to-process-a-dataframe-in-parallel-make-pandas-lightning-fast-669978cf5356

logical    = False
df_results = []
num_procs  = psutil.cpu_count(logical=logical)
#if len(sys.argv) > 1:
#    num_procs = int(sys.argv[1])

big_dataframe = x.copy()
splitted_df = np.array_split(big_dataframe, num_procs)
start = time.time()
with concurrent.futures.ProcessPoolExecutor(max_workers=num_procs) as executor:
    results = [ executor.submit(do_something_2, x=df) for df in splitted_df ]
    for result in concurrent.futures.as_completed(results):
        try:
            df_results.append(result.result())
        except Exception as ex:
            print(str(ex))
            pass
end = time.time()
print("-------------------------------------------")
print("PPID %s Completed in %s"%(os.getpid(), round(end-start,2)))
df_results = pd.concat(df_results)

PPID 75->329 Started
-------------------------------------------
PPID 75 Completed in 10.51


In [30]:
x = df_results.copy()
x.to_csv('cna_scaled_normailized.csv', index=False)
x.head()

,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,...,0.002546,0.002546,0.002583,0.002498,0.002571,0.002477,0.002477,0.002477,0.002477,0.002473
1,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,...,0.002546,0.002546,0.002583,0.002498,0.002571,0.002477,0.002477,0.002477,0.002477,0.002473
2,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.001334,0.000235,0.001308,0.000234,0.001297,0.001297,0.001297,0.001297,0.000225
3,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.001334,0.000235,0.001308,0.000234,0.002477,0.002477,0.002477,0.002477,0.000225
4,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.001334,0.000235,0.001308,0.000234,0.001297,0.001297,0.001297,0.001297,0.000225


## DNA

In [32]:
df_dna = pd.read_csv('./DNA-TMB.csv')
df_dna.head()

,SAMPLE_ID,ATP2A1_NM_173201_NM_004320_cg00000292,MEOX2_NM_005924_cg00003994,HOXD3_NM_006898_cg00005847,PANX1_NM_015368_cg00007981,KIAA1409_COX8C_NM_020818_NM_182971_cg00008493,IMPA2_NM_014214_cg00008713,TTC8_NM_144596_NM_198310_NM_198309_cg00009407,PMM2_TMEM186_NM_000303_NM_015421_cg00011459,ANG_RNASE4_NM_002937_NM_001145_cg00012199,...,ALDH1A3_NM_000693_cg27652350,A2ML1_NM_144670_cg27653134,ZFP2_NM_030613_cg27654142,C11orf24_NM_022338_cg27655905,LEPRE1_C1orf50_NM_022356_NM_001146289_NM_024097_cg27657283,GNAS_NM_016592_NM_001077490_NM_080425_cg27661264,RPN1_NM_002950_cg27662379,CYB5A_NM_001914_NM_148923_cg27662877,AP1S1_NM_001283_cg27665659,TMB_CLASS
0,TCGA-2F-A9KO-01,0.680818,0.062263,0.496348,0.028165,0.902485,0.049487,0.030269,0.903082,0.033687,...,0.226244,0.493849,0.257453,0.032806,0.037936,0.249382,0.042351,0.034030,0.051435,2
1,TCGA-2F-A9KP-01,0.582509,0.137747,0.632550,0.023012,0.943398,0.052264,0.029266,0.899903,0.034254,...,0.692096,0.344149,0.091569,0.033999,0.027818,0.163510,0.031568,0.035252,0.056094,1
2,TCGA-2F-A9KQ-01,0.494600,0.045964,0.894574,0.020186,0.935219,0.042077,0.024276,0.902709,0.035322,...,0.255991,0.204110,0.028300,0.027538,0.027321,0.341883,0.038592,0.036476,0.032838,1
3,TCGA-2F-A9KR-01,0.428810,0.049838,0.837669,0.020671,0.947272,0.048717,0.027192,0.778382,0.029718,...,0.753269,0.183719,0.035366,0.038756,0.025200,0.190627,0.040013,0.037121,0.040571,2
4,TCGA-2F-A9KT-01,0.473822,0.043334,0.580027,0.024262,0.944825,0.043531,0.028673,0.919768,0.029490,...,0.591642,0.450381,0.049927,0.037207,0.022635,0.313394,0.045062,0.045077,0.039392,2


In [33]:
x = df_dna.drop(['SAMPLE_ID', 'TMB_CLASS'], axis=1)
x.head()

,ATP2A1_NM_173201_NM_004320_cg00000292,MEOX2_NM_005924_cg00003994,HOXD3_NM_006898_cg00005847,PANX1_NM_015368_cg00007981,KIAA1409_COX8C_NM_020818_NM_182971_cg00008493,IMPA2_NM_014214_cg00008713,TTC8_NM_144596_NM_198310_NM_198309_cg00009407,PMM2_TMEM186_NM_000303_NM_015421_cg00011459,ANG_RNASE4_NM_002937_NM_001145_cg00012199,SNAP47_JMJD4_NM_053052_NM_001161465_NM_023007_cg00012386,...,KIAA1409_COX8C_NM_020818_NM_182971_cg27651218,ALDH1A3_NM_000693_cg27652350,A2ML1_NM_144670_cg27653134,ZFP2_NM_030613_cg27654142,C11orf24_NM_022338_cg27655905,LEPRE1_C1orf50_NM_022356_NM_001146289_NM_024097_cg27657283,GNAS_NM_016592_NM_001077490_NM_080425_cg27661264,RPN1_NM_002950_cg27662379,CYB5A_NM_001914_NM_148923_cg27662877,AP1S1_NM_001283_cg27665659
0,0.680818,0.062263,0.496348,0.028165,0.902485,0.049487,0.030269,0.903082,0.033687,0.035757,...,0.901627,0.226244,0.493849,0.257453,0.032806,0.037936,0.249382,0.042351,0.034030,0.051435
1,0.582509,0.137747,0.632550,0.023012,0.943398,0.052264,0.029266,0.899903,0.034254,0.029549,...,0.641527,0.692096,0.344149,0.091569,0.033999,0.027818,0.163510,0.031568,0.035252,0.056094
2,0.494600,0.045964,0.894574,0.020186,0.935219,0.042077,0.024276,0.902709,0.035322,0.042278,...,0.877061,0.255991,0.204110,0.028300,0.027538,0.027321,0.341883,0.038592,0.036476,0.032838
3,0.428810,0.049838,0.837669,0.020671,0.947272,0.048717,0.027192,0.778382,0.029718,0.034014,...,0.874384,0.753269,0.183719,0.035366,0.038756,0.025200,0.190627,0.040013,0.037121,0.040571
4,0.473822,0.043334,0.580027,0.024262,0.944825,0.043531,0.028673,0.919768,0.029490,0.055943,...,0.746931,0.591642,0.450381,0.049927,0.037207,0.022635,0.313394,0.045062,0.045077,0.039392


In [35]:
y = df_dna[['TMB_CLASS']]
y.head()

,TMB_CLASS
0,2
1,1
2,1
3,2
4,2


In [36]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
x_df_scaled = pd.DataFrame(x_scaled, columns=x.columns)
x_df_scaled.head()

,ATP2A1_NM_173201_NM_004320_cg00000292,MEOX2_NM_005924_cg00003994,HOXD3_NM_006898_cg00005847,PANX1_NM_015368_cg00007981,KIAA1409_COX8C_NM_020818_NM_182971_cg00008493,IMPA2_NM_014214_cg00008713,TTC8_NM_144596_NM_198310_NM_198309_cg00009407,PMM2_TMEM186_NM_000303_NM_015421_cg00011459,ANG_RNASE4_NM_002937_NM_001145_cg00012199,SNAP47_JMJD4_NM_053052_NM_001161465_NM_023007_cg00012386,...,KIAA1409_COX8C_NM_020818_NM_182971_cg27651218,ALDH1A3_NM_000693_cg27652350,A2ML1_NM_144670_cg27653134,ZFP2_NM_030613_cg27654142,C11orf24_NM_022338_cg27655905,LEPRE1_C1orf50_NM_022356_NM_001146289_NM_024097_cg27657283,GNAS_NM_016592_NM_001077490_NM_080425_cg27661264,RPN1_NM_002950_cg27662379,CYB5A_NM_001914_NM_148923_cg27662877,AP1S1_NM_001283_cg27665659
0,0.685991,0.044830,0.435125,0.500103,0.858026,0.391915,0.313802,0.867840,0.031857,0.288657,...,0.873366,0.180542,0.472144,0.376256,0.198893,0.186756,0.244477,0.167952,0.024768,0.223838
1,0.556211,0.129177,0.603061,0.327710,0.953919,0.431897,0.289914,0.861327,0.033141,0.194665,...,0.418154,0.737618,0.282977,0.106990,0.215740,0.108160,0.120736,0.063156,0.027021,0.258723
2,0.440161,0.026617,0.926135,0.233180,0.934749,0.285231,0.171023,0.867075,0.035562,0.387378,...,0.830371,0.216113,0.106019,0.004290,0.124502,0.104297,0.377772,0.131427,0.029279,0.084571
3,0.353311,0.030946,0.855972,0.249420,0.962998,0.380832,0.240509,0.612413,0.022858,0.262262,...,0.825687,0.810771,0.080251,0.015758,0.282911,0.087821,0.159811,0.145232,0.030468,0.142483
4,0.412732,0.023678,0.538301,0.369558,0.957264,0.306161,0.275775,0.902018,0.022343,0.594269,...,0.602626,0.617493,0.417216,0.039395,0.261035,0.067901,0.336720,0.194296,0.045142,0.133653


In [37]:
x_df_scaled.to_csv('./dna_scaled_x.csv', index=False)

## mRNA

In [39]:
df_mrna = pd.read_csv('./mRNA-TMB.csv')
df_mrna.head()

,SAMPLE_ID,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,TMB_CLASS
0,TCGA-2F-A9KO-01,37.8717,123.090,0.5831,0.0,11.0787,2.3324,4.0816,0.0000,0.0,...,554.519,102.6240,776.676,1450.730,5.8309,697.959,4262.39,1787.760,572.595,2
1,TCGA-2F-A9KP-01,12.5759,137.886,0.8323,0.0,0.0000,4.5776,2.0807,0.4161,0.0,...,969.621,37.0370,316.688,1420.720,17.0620,1526.430,3105.29,1467.750,860.591,1
2,TCGA-2F-A9KQ-01,10.7742,104.678,0.0000,0.0,0.0000,0.4636,2.3180,0.0000,0.0,...,795.086,60.2689,378.303,873.435,1.8544,1571.160,3275.38,750.580,621.233,1
3,TCGA-2F-A9KR-01,42.5810,146.453,2.3743,0.0,1.8995,1.4246,3.7989,0.0000,0.0,...,568.412,45.5869,470.115,1924.150,1.8995,748.860,4465.62,910.788,935.481,2
4,TCGA-2F-A9KT-01,17.6427,142.962,0.0000,0.0,5.0876,2.2612,6.2182,0.0000,0.0,...,1087.050,17.5240,281.515,938.383,6.7835,878.462,3449.41,850.198,630.865,2


In [40]:
x = df_mrna.drop(['SAMPLE_ID', 'TMB_CLASS'], axis=1)
x.head()

,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,REXO1L6P,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,37.8717,123.090,0.5831,0.0,11.0787,2.3324,4.0816,0.0000,0.0,0.0,...,694.857,554.519,102.6240,776.676,1450.730,5.8309,697.959,4262.39,1787.760,572.595
1,12.5759,137.886,0.8323,0.0,0.0000,4.5776,2.0807,0.4161,0.0,0.0,...,718.119,969.621,37.0370,316.688,1420.720,17.0620,1526.430,3105.29,1467.750,860.591
2,10.7742,104.678,0.0000,0.0,0.0000,0.4636,2.3180,0.0000,0.0,0.0,...,532.536,795.086,60.2689,378.303,873.435,1.8544,1571.160,3275.38,750.580,621.233
3,42.5810,146.453,2.3743,0.0,1.8995,1.4246,3.7989,0.0000,0.0,0.0,...,616.848,568.412,45.5869,470.115,1924.150,1.8995,748.860,4465.62,910.788,935.481
4,17.6427,142.962,0.0000,0.0,5.0876,2.2612,6.2182,0.0000,0.0,0.0,...,726.625,1087.050,17.5240,281.515,938.383,6.7835,878.462,3449.41,850.198,630.865


In [42]:
y = df_mrna[['TMB_CLASS']]
y.head()

,TMB_CLASS
0,2
1,1
2,1
3,2
4,2


In [43]:
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
x_df_scaled = pd.DataFrame(x_scaled, columns=x.columns)
x_df_scaled.head()

,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,REXO1L6P,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.435705,0.186240,0.127043,0.0,0.004761,0.015401,0.165885,0.000000,0.0,0.0,...,0.273604,0.140067,0.643404,0.608620,0.434340,0.007538,0.266814,0.226097,0.495467,0.084136
1,0.144683,0.215082,0.181337,0.0,0.000000,0.030226,0.084564,0.034472,0.0,0.0,...,0.283346,0.248508,0.229734,0.244889,0.423281,0.022057,0.709348,0.161651,0.392767,0.148803
2,0.123955,0.150349,0.000000,0.0,0.000000,0.003061,0.094208,0.000000,0.0,0.0,...,0.205628,0.202912,0.376262,0.293611,0.221606,0.002397,0.733241,0.171124,0.162607,0.095057
3,0.489884,0.231782,0.517299,0.0,0.000816,0.009407,0.154395,0.000000,0.0,0.0,...,0.240936,0.143696,0.283660,0.366210,0.608795,0.002456,0.294003,0.237416,0.214022,0.165619
4,0.202975,0.224977,0.000000,0.0,0.002187,0.014931,0.252721,0.000000,0.0,0.0,...,0.286908,0.279185,0.106662,0.217076,0.245539,0.008770,0.363231,0.180817,0.194577,0.097220


In [44]:
x_df_scaled.to_csv('./mrna_scaled_x.csv', index=False)

## putting engineered data together to upload on github

In [45]:
!tar zcf omic_engineered_data_x.tar.gz ./cna_scaled_normailized.csv ./mrna_scaled_x.csv ./dna_scaled_x.csv

In [49]:
# recreate updated dataframes
x_cna = pd.read_csv('./cna_scaled_normailized.csv')
df_cna_orig = pd.read_csv('./CNA-TMB.csv')
df_cna_final = x_cna.copy()
df_cna_final.insert(0, column='SAMPLE_ID', value=df_cna_orig['SAMPLE_ID'])
df_cna_final['TMB_CLASS'] = df_cna_orig['TMB_CLASS'].copy()
df_cna_final.to_csv('cna_engineered.csv', index=False)
df_cna_final.head()

,SAMPLE_ID,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224,TMB_CLASS
0,TCGA-2F-A9KO-01,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,0.001260,...,0.002546,0.002583,0.002498,0.002571,0.002477,0.002477,0.002477,0.002477,0.002473,2
1,TCGA-2F-A9KP-01,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,0.004698,...,0.002546,0.002583,0.002498,0.002571,0.002477,0.002477,0.002477,0.002477,0.002473,1
2,TCGA-2F-A9KQ-01,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.000235,0.001308,0.000234,0.001297,0.001297,0.001297,0.001297,0.000225,1
3,TCGA-2F-A9KR-01,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.000235,0.001308,0.000234,0.002477,0.002477,0.002477,0.002477,0.000225,2
4,TCGA-2F-A9KT-01,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,0.002406,...,0.001334,0.000235,0.001308,0.000234,0.001297,0.001297,0.001297,0.001297,0.000225,2


In [50]:
x_dna = pd.read_csv('./dna_scaled_x.csv')
df_dna_orig = pd.read_csv('./DNA-TMB.csv')
df_dna_final = x_dna.copy()
df_dna_final.insert(0, column='SAMPLE_ID', value=df_dna_orig['SAMPLE_ID'])
df_dna_final['TMB_CLASS'] = df_dna_orig['TMB_CLASS'].copy()
df_dna_final.to_csv('dna_engineered.csv', index=False)
df_dna_final.head()

,SAMPLE_ID,ATP2A1_NM_173201_NM_004320_cg00000292,MEOX2_NM_005924_cg00003994,HOXD3_NM_006898_cg00005847,PANX1_NM_015368_cg00007981,KIAA1409_COX8C_NM_020818_NM_182971_cg00008493,IMPA2_NM_014214_cg00008713,TTC8_NM_144596_NM_198310_NM_198309_cg00009407,PMM2_TMEM186_NM_000303_NM_015421_cg00011459,ANG_RNASE4_NM_002937_NM_001145_cg00012199,...,ALDH1A3_NM_000693_cg27652350,A2ML1_NM_144670_cg27653134,ZFP2_NM_030613_cg27654142,C11orf24_NM_022338_cg27655905,LEPRE1_C1orf50_NM_022356_NM_001146289_NM_024097_cg27657283,GNAS_NM_016592_NM_001077490_NM_080425_cg27661264,RPN1_NM_002950_cg27662379,CYB5A_NM_001914_NM_148923_cg27662877,AP1S1_NM_001283_cg27665659,TMB_CLASS
0,TCGA-2F-A9KO-01,0.685991,0.044830,0.435125,0.500103,0.858026,0.391915,0.313802,0.867840,0.031857,...,0.180542,0.472144,0.376256,0.198893,0.186756,0.244477,0.167952,0.024768,0.223838,2
1,TCGA-2F-A9KP-01,0.556211,0.129177,0.603061,0.327710,0.953919,0.431897,0.289914,0.861327,0.033141,...,0.737618,0.282977,0.106990,0.215740,0.108160,0.120736,0.063156,0.027021,0.258723,1
2,TCGA-2F-A9KQ-01,0.440161,0.026617,0.926135,0.233180,0.934749,0.285231,0.171023,0.867075,0.035562,...,0.216113,0.106019,0.004290,0.124502,0.104297,0.377772,0.131427,0.029279,0.084571,1
3,TCGA-2F-A9KR-01,0.353311,0.030946,0.855972,0.249420,0.962998,0.380832,0.240509,0.612413,0.022858,...,0.810771,0.080251,0.015758,0.282911,0.087821,0.159811,0.145232,0.030468,0.142483,2
4,TCGA-2F-A9KT-01,0.412732,0.023678,0.538301,0.369558,0.957264,0.306161,0.275775,0.902018,0.022343,...,0.617493,0.417216,0.039395,0.261035,0.067901,0.336720,0.194296,0.045142,0.133653,2


In [51]:
x_mrna = pd.read_csv('./mrna_scaled_x.csv')
df_mrna_orig = pd.read_csv('./mRNA-TMB.csv')
df_mrna_final = x_mrna.copy()
df_mrna_final.insert(0, column='SAMPLE_ID', value=df_mrna_orig['SAMPLE_ID'])
df_mrna_final['TMB_CLASS'] = df_dna_orig['TMB_CLASS'].copy()
df_mrna_final.to_csv('mrna_engineered.csv', index=False)
df_mrna_final.head()

,SAMPLE_ID,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,TMB_CLASS
0,TCGA-2F-A9KO-01,0.435705,0.186240,0.127043,0.0,0.004761,0.015401,0.165885,0.000000,0.0,...,0.140067,0.643404,0.608620,0.434340,0.007538,0.266814,0.226097,0.495467,0.084136,2
1,TCGA-2F-A9KP-01,0.144683,0.215082,0.181337,0.0,0.000000,0.030226,0.084564,0.034472,0.0,...,0.248508,0.229734,0.244889,0.423281,0.022057,0.709348,0.161651,0.392767,0.148803,1
2,TCGA-2F-A9KQ-01,0.123955,0.150349,0.000000,0.0,0.000000,0.003061,0.094208,0.000000,0.0,...,0.202912,0.376262,0.293611,0.221606,0.002397,0.733241,0.171124,0.162607,0.095057,1
3,TCGA-2F-A9KR-01,0.489884,0.231782,0.517299,0.0,0.000816,0.009407,0.154395,0.000000,0.0,...,0.143696,0.283660,0.366210,0.608795,0.002456,0.294003,0.237416,0.214022,0.165619,2
4,TCGA-2F-A9KT-01,0.202975,0.224977,0.000000,0.0,0.002187,0.014931,0.252721,0.000000,0.0,...,0.279185,0.106662,0.217076,0.245539,0.008770,0.363231,0.180817,0.194577,0.097220,2


In [52]:
# compressing them to be used later
!tar zcf omic-enigneered-df.tar.gz ./cna_engineered.csv ./dna_engineered.csv ./mrna_engineered.csv

In [59]:
c = [True,False,True,False,True,True,True,False]
c = [1, 0 , 1 , 0, 1 , 1, 1, 0]
vector_binary = [bool(item == 1) for item in c]
print(vector_binary)
df_cna_final = df_cna_final.iloc[:, :8]
df_cna_final[df_cna_final.columns[vector_binary]].head()

[True, False, True, False, True, True, True, False]


,SAMPLE_ID,ACTRT2,ANKRD65,ATAD3A,ATAD3B
0,TCGA-2F-A9KO-01,0.001260,0.001260,0.001260,0.001260
1,TCGA-2F-A9KP-01,0.004698,0.004698,0.004698,0.004698
2,TCGA-2F-A9KQ-01,0.002406,0.002406,0.002406,0.002406
3,TCGA-2F-A9KR-01,0.002406,0.002406,0.002406,0.002406
4,TCGA-2F-A9KT-01,0.002406,0.002406,0.002406,0.002406


In [77]:
'''
data = [1,2,2,3,3,4]

pd_series = df_mrna_final['TMB_CLASS'].mean()
print(pd_series)
counts = pd_series.value_counts()
print(counts)
'''

"\ndata = [1,2,2,3,3,4]\n\npd_series = df_mrna_final['TMB_CLASS'].mean()\nprint(pd_series)\ncounts = pd_series.value_counts()\nprint(counts)\n"

# Defining the cost function

In [70]:
# credit to https://winder.ai/entropy-based-feature-selection/

def entropy(y):
    probs = [] # Probabilities of each class label
    for c in set(y): # Set gets a unique set of values. We're iterating over each value
        num_same_class = sum(y == c)  # Remember that true == 1, so we can sum.
        p = num_same_class / len(y) # Probability of this class label
        probs.append(p)
    return np.sum(-p * np.log2(p) for p in probs)

def class_probability(feature, y):
  probs = []
  split_point = feature.mean()
  feature = np.where(feature < split_point, 0, 1)
  for value in set(feature):
        select = feature == value # Split by feature value into two classes
        y_new = y[select]         # Those that exist in this class are now in y_new
        probs.append(float(len(y_new))/len(feature))  # Convert to float, because ints don't divide well
  return probs

def class_entropy(feature, y):
    ents = []
    split_point = feature.mean()
    feature = np.where(feature < split_point, 0, 1)
    for value in set(feature):
        select = feature == value # Split by feature value into two classes
        y_new = y[select]         # Those that exist in this class are now in y_new
        ents.append(entropy(y_new))
    return ents

def proportionate_class_entropy(feature, y):
    """Calculatates the weighted proportional entropy for a feature when splitting on all values"""
    probs = class_probability(feature, y)
    ents = class_entropy(feature, y)
    return np.sum(np.multiply(probs, ents)) # Information gain equation

In [75]:
'''
# testing cell
for i in df_mrna_final.iloc[:, 1:600] :
  print(i)
  print(proportionate_class_entropy(df_mrna_final[i], df_mrna_final['TMB_CLASS']))
'''

"\n# testing cell\nfor i in df_mrna_final.iloc[:, 1:600] :\n  print(i)\n  print(proportionate_class_entropy(df_mrna_final[i], df_mrna_final['TMB_CLASS']))\n"

In [76]:
def costFun(vector, df):
  y = df['TMB_CLASS']
  vector_binary = [bool(item == 1) for item in vector]
  x = df[df.columns[vector_binary]].copy()
  cumulative_entropy=0
  for feature in x.columns:
    cumulative_entropy += proportionate_class_entropy(feature, y)
  
  return cumulative_entropy


# Starting with GA

## downloading engineered data